In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import *
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator, Options

# This script is for extracting the probability distributions with/without different
# error reduction techniques from IBM hardware. 7 Qubits limits it to max 6 generals.
# These probability distributions are then sampled from in the full scheme.

# This can be achieved by altering the Optimization level, and Resilience level.
# Optimization level=3 is dynamical decoupling.
# Resilience level=1 is T-REx error mitigation.

# service.backends() # 'ibm_nairobi',  'ibmq_qasm_simulator'

In [ ]:
# Creating starting states
states = []
for ns in range(3,7):
    state = np.zeros(2**(ns+1))
    state[2**(ns-1)-1] = np.sqrt(1/3)  # Retreat order from commander
    state[2**(ns) + 2**(ns-1)] = np.sqrt(1/3)  # Attack order from commander
    for i in range(ns-1):
        state[2**(ns-1) + 2**i] = np.sqrt(1/(6*ns-6))  # commander '01'
        state[2**(ns) + 2**(ns-1) - 1 - 2**i] = np.sqrt(1/(6*ns-6))  # commander '10'
    states.append(state)

# Creating Circuits
circuits = []
for qbs in range(4,8): # looping over number of qubits
    circuit = QuantumCircuit(qbs)
    circuit.initialize(states[qbs-4], circuit.qubits)
    circuit.measure_all()
    circuits.append(circuit)



In [ ]:
results = []
repeats = 100
service = QiskitRuntimeService()
print('starting session')
with Session(service=service, backend='ibm_nairobi') as session:
    sampler1 = Sampler(session=session, options=Options(optimization_level=0, resilience_level=0))
    sampler2 = Sampler(session=session, options=Options(optimization_level=0, resilience_level=1))
    sampler3 = Sampler(session=session, options=Options(optimization_level=3, resilience_level=0))
    sampler4 = Sampler(session=session, options=Options(optimization_level=3, resilience_level=1))
    print('running jobs')
    job1 = sampler1.run(repeats*circuits, shots=8192).result().quasi_dists
    job2 = sampler2.run(repeats*circuits, shots=8192).result().quasi_dists
    job3 = sampler3.run(repeats*circuits, shots=8192).result().quasi_dists
    job4 = sampler4.run(repeats*circuits, shots=8192).result().quasi_dists
    print('saving results')
    results.append([qd.nearest_probability_distribution() for qd in job1])
    results.append([qd.nearest_probability_distribution() for qd in job2])
    results.append([qd.nearest_probability_distribution() for qd in job3])
    results.append([qd.nearest_probability_distribution() for qd in job4])
    session.close()

# open file in write mode
with open('results.txt', 'w+') as fp:
    for item in results:  # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')


In [ ]:
# Function to split by number of generals
def splitter(opt_res):
    gen3, gen4, gen5, gen6 = [], [], [], []
    for i in range(len(opt_res)):
        if i%4 == 0:
            gen3.append(opt_res[i])
        elif i%4 == 1:
            gen4.append(opt_res[i])
        elif i%4 == 2:
            gen5.append(opt_res[i])
        else:
            gen6.append(opt_res[i])
    return gen3, gen4, gen5, gen6


# Function to reshape each list of 100 results
def reshaper(gen_res, n):
    new_gen_res = []
    for result in gen_res:
        bins, probs = [], []
        for k,v in result.items():
            bins.append(format(k, '0'+str(n+1)+'b'))
            probs.append(v)
        new_gen_res.append((bins, probs))
    return new_gen_res

In [ ]:
# Loading results
results = open('results.txt', "r").readlines()  # length = 4

# Reformatting
new_results = []
for samp in results:
    var = eval(samp)  # Contains 4 subdicts for 4 different opt/res combiniations
    new_results.append(var)  # Each sublist has length = 400

# splitting
gen3_00, gen4_00, gen5_00, gen6_00 = splitter(new_results[0])
gen3_01, gen4_01, gen5_01, gen6_01 = splitter(new_results[1])
gen3_30, gen4_30, gen5_30, gen6_30 = splitter(new_results[2])
gen3_31, gen4_31, gen5_31, gen6_31 = splitter(new_results[3])

# reshaping
gen3_00 = reshaper(gen3_00, 3)
gen3_01 = reshaper(gen3_01, 3)
gen3_30 = reshaper(gen3_30, 3)
gen3_31 = reshaper(gen3_31, 3)

gen4_00 = reshaper(gen4_00, 4)
gen4_01 = reshaper(gen4_01, 4)
gen4_30 = reshaper(gen4_30, 4)
gen4_31 = reshaper(gen4_31, 4)

gen5_00 = reshaper(gen5_00, 5)
gen5_01 = reshaper(gen5_01, 5)
gen5_30 = reshaper(gen5_30, 5)
gen5_31 = reshaper(gen5_31, 5)

gen6_00 = reshaper(gen6_00, 6)
gen6_01 = reshaper(gen6_01, 6)
gen6_30 = reshaper(gen6_30, 6)
gen6_31 = reshaper(gen6_31, 6)

results = [[gen3_00, gen3_01, gen3_30, gen3_31], [gen4_00, gen4_01, gen4_30, gen4_31],
          [gen5_00, gen5_01, gen5_30, gen5_31], [gen6_00, gen6_01, gen6_30, gen6_31]]

# open file in write mode
with open('results_reformatted.txt', 'w+') as fp:
    for gen_num in results:
        for opt_res in gen_num:
            fp.write("%s\n" % opt_res)
    print('Done')